![](https://s14-eu5.ixquick.com/cgi-bin/serveimage?url=https%3A%2F%2Fwww.alojandoturistas.com%2Fwp-content%2Fuploads%2F2016%2F03%2Finsideairbnb01.jpg&sp=98223f3b39fbea79a7ea767e03a1cf15)

## INTRODUCTION

The following project is based on a dataset found [here](http://insideairbnb.com/get-the-data.html), an independent service not
related to Airbnb that offers Airbnb listings data for free in  major cities of the world.

The dataset is a picture of the airbnb website, but is not Airbnb private data. It has been collected through webscrapping.

Airbnb website is free to list any property, this involves that some listings are not real. After thorough exploration of 
the data, we realised that many listings were just fake, other were not online anymore, and sometimes they were not even homes. Many appear in Airbnb website  but they are not available for rent. 

Consequently, the first stage of this project consists of finding all those fake listings and delete them in order to 
have an accurate sample of the vacation rentals occupancy rate in Madrid



## Find and remove fake and inactive listings. 
  There are three types of inactive properties: 

    a) Listings that have never been available (although they turn up the in website).
    
    b) Listings that were active in the past but for some reason they are not available any more.
    
    c) Semi active: properties only available a few days a month(ie, only weekends) or canceled by the host
    that removes on purpose all availability during the month.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
# File available in dataset folder
aib = pd.read_csv('listings.csv.gz')
aib.shape

(13335, 95)


## Filters  
The most important elements to measure if a property is available for rent full time and the listing is real are the following variables(columns):

 1) Number of reviews.
 
 2) Reviews per month.
 
 3) Calendar last updated.
 
 4) Availability_30/60/90.
 
 5) Last_review // First_review
 



## Filter 1
Rooms/apartments that appear sold out in the next 3 months (availability_30=0 & availability_60=0 & availability_90=0)
and  have no reviews at all, thus "reviews per month" and "number of reviews" columns equal to 0 or NaN.


In [6]:
# Find and delete
aib = aib.drop(aib[(aib.number_of_reviews==0) & (aib['reviews_per_month'].isnull()) & (aib.availability_30==0) & 
      (aib.availability_60==0)  & (aib.availability_90==0)].index)
aib.shape

(12662, 95)

## Filter 2
Properties not updated. Properties listed for more than a year,having less than 11 reviews with extreme low availability.

In [7]:

aib['last_review'] = pd.to_datetime(aib.last_review,format='%Y-%m-%d')
aib['first_review'] = pd.to_datetime(aib.first_review,format='%Y-%m-%d')

# Dataset as of April 2017.
aib =aib.drop(aib[(aib.number_of_reviews<11) & (aib.first_review<'2016-03-01') & (aib.availability_60<3) 
    & (aib.availability_90<5)].index)
aib.shape

(12293, 95)

In [8]:

aib.reviews_per_month.mean()


1.98623316622432

## Filter 3
Properties with very low availability, reviews per month well below the average (1.98) and last review in 2016.

In [9]:
aib = aib.drop(aib[(aib.availability_30<2) & (aib.availability_60<5) & (aib.reviews_per_month<1.5) 
    & (aib.last_review<'2017-01-01')].index)
aib.shape

(11557, 95)

In [10]:
# Final filter based on number of reviews along with 'reviews per month'
aib =  aib.drop(aib[(aib.availability_30==0) & (aib.number_of_reviews<=7) 
    & (aib.reviews_per_month<1.5)].index)
aib.shape

(11112, 95)

In [11]:
aib =  aib.drop(aib[(aib.availability_30==0)  & (aib.number_of_reviews<=7) 
    & (aib.reviews_per_month.isnull())].index)
aib.shape

(10687, 95)

## Filter 4.
Listings showing misleading availability.

Some properties turn up with average "number of reviews", "reviews per month" and very low availability. They look fine but thay are not. In fact, their availability is not real, it has been canceled by the host so that no one can book the property in the current month. 

These kind of offerings are the hardest to spot, so to reduce the error margin we establish a more accurate and 
customized filter. Each listing with no availability_30 must show a number of reviews greater than
the neighbourhood average.
We assume that if a property is fully booked, it is very popular and therefore, it must show more
reviews than regular properties. 

In [12]:
# Find out average number of reviews of every district
aib.groupby('neighbourhood_group_cleansed').number_of_reviews.mean()

neighbourhood_group_cleansed
Arganzuela               24.296596
Barajas                  38.196970
Carabanchel              12.534483
Centro                   37.414127
Chamartín                15.654275
Chamberí                 18.672154
Ciudad Lineal            14.903226
Fuencarral - El Pardo     7.570423
Hortaleza                13.748428
Latina                   17.061433
Moncloa - Aravaca        13.313783
Moratalaz                15.176471
Puente de Vallecas       16.708333
Retiro                   22.683908
Salamanca                15.916306
San Blas - Canillejas    22.623762
Tetuán                   15.387931
Usera                    13.794872
Vicálvaro                 3.210526
Villa de Vallecas         7.857143
Villaverde               15.514706
Name: number_of_reviews, dtype: float64

In [13]:
# Filter applied
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Arganzuela') & (aib.number_of_reviews<=24)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Barajas') & (aib.number_of_reviews<=38)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Carabanchel') & (aib.number_of_reviews<=12)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Centro') & (aib.number_of_reviews<=37)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Chamartín') & (aib.number_of_reviews<=15)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Chamberí') & (aib.number_of_reviews<=18)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Ciudad Lineal') & (aib.number_of_reviews<=14)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Fuencarral - El Pardo') & (aib.number_of_reviews<=7)
    & (aib.availability_30==0)].index)
aib.shape

(10315, 95)

In [14]:
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Hortaleza') & (aib.number_of_reviews<=13)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Latina') & (aib.number_of_reviews<=17)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Moratalaz') & (aib.number_of_reviews<=15)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Moncloa - Aravaca') & (aib.number_of_reviews<=13)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Puente de Vallecas') & (aib.number_of_reviews<=16)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Retiro') & (aib.number_of_reviews<=22)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Salamanca') & (aib.number_of_reviews<=15)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='San Blas - Canillejas') & (aib.number_of_reviews<=22)
    & (aib.availability_30==0)].index)
aib.shape

(10236, 95)

In [15]:
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Tetuan') & (aib.number_of_reviews<=15)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Usera') & (aib.number_of_reviews<=13)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Vicálvaro') & (aib.number_of_reviews<=3)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Villa de Vallecas') & (aib.number_of_reviews<=7)
    & (aib.availability_30==0)].index)
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Villaverde') & (aib.number_of_reviews<=15)
    & (aib.availability_30==0)].index)
aib.shape


(10231, 95)

## Check availability_30 distribution. 
### Additional Filters

In [ ]:
sns.distplot(aib.availability_30)
sns.plt.show()

In [ ]:
sns.distplot(aib.availability_60)
sns.plt.show()

In [ ]:
sns.distplot(aib.availability_90)
sns.plt.show()

In [16]:
# Charts are still showing high frequency of listings with no availability in 3 months.
# Full occupancy during three or two  months is extremly rare so additional filtering is required.
ai9 = aib[(aib.availability_90==0)]
ai9[['id','price','neighbourhood_cleansed','number_of_reviews','reviews_per_month','host_since','last_review']]

,id,price,neighbourhood_cleansed,number_of_reviews,reviews_per_month,host_since,last_review
237,10064795,$149.00,Palacio,65,4.28,2015-12-18,2017-02-14
244,3113226,$60.00,Palacio,53,1.59,2014-05-21,2017-01-10
573,5499969,$35.00,Palacio,47,1.89,2015-02-28,2017-01-30
577,5860795,$15.00,Palacio,56,2.41,2014-05-05,2017-02-19
652,11078225,$45.00,Palacio,57,4.02,2015-09-15,2017-02-05
666,4748729,$60.00,Palacio,58,2.10,2012-04-15,2016-06-05
901,4196358,$80.00,Palacio,46,1.50,2014-01-20,2017-03-13
1329,4138837,$55.00,Embajadores,71,2.31,2014-09-18,2017-01-13
1423,268275,$72.00,Embajadores,42,0.66,2011-11-14,2017-01-22
1472,4755562,$50.00,Embajadores,48,1.81,2014-12-06,2017-03-26


In [17]:
# There are still many inactive listings. It is remarkable those with no availability in 90 days and last review in 2016.
# We get rid of those listings.
# High percentage of listings show high occupancy but not full. Maybe this is due to the season,hence
# availability_90 is not a reliable variable
aib = aib.drop(aib[(aib.availability_90==0)&(aib.last_review<'2017-01-01')].index)
aib.shape

(10202, 95)

In [18]:
# Same with 60 days
aib10 = aib[(aib.availability_60==0)]
aib10[['id','availability_30','neighbourhood_group_cleansed','number_of_reviews','reviews_per_month','room_type','host_since','last_review']]

,id,availability_30,neighbourhood_group_cleansed,number_of_reviews,reviews_per_month,room_type,host_since,last_review
19,6617552,0,Centro,51,2.38,Private room,2015-05-31,2017-01-09
237,10064795,0,Centro,65,4.28,Entire home/apt,2015-12-18,2017-02-14
244,3113226,0,Centro,53,1.59,Entire home/apt,2014-05-21,2017-01-10
289,761015,0,Centro,40,0.77,Entire home/apt,2012-06-18,2017-01-01
345,5974127,0,Centro,92,3.91,Private room,2015-04-11,2017-02-27
507,9089327,0,Centro,40,2.29,Entire home/apt,2015-10-25,2017-02-27
573,5499969,0,Centro,47,1.89,Private room,2015-02-28,2017-01-30
577,5860795,0,Centro,56,2.41,Private room,2014-05-05,2017-02-19
591,1765529,0,Centro,40,1.09,Entire home/apt,2013-10-07,2017-03-24
652,11078225,0,Centro,57,4.02,Entire home/apt,2015-09-15,2017-02-05


In [19]:
# Manual verification of previous query reveals that majority of listings do not exist anymore or are inactive.
# Therefore we remove all properties NOT located in the city center showing availability_30 and 
# availability_60 equal to 0.

aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Arganzuela')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Tetuán')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Chamartín')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Retiro')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='San Blas - Canillejas')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Moratalaz')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Barajas')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Moncloa - Aravaca')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Latina')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Fuencarral - El Pardo')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Carabanchel')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Usera')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Puente de Vallecas')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Ciudad Lineal')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Hortaleza')].index)
aib = aib.drop(aib[(aib.availability_30==0)&(aib.availability_60==0)&(aib.neighbourhood_group_cleansed=='Villaverde')].index)
aib.shape


(10120, 95)

In [ ]:
sns.distplot(aib.availability_60)
sns.plt.show()

In [ ]:
sns.distplot(aib.availability_90)
sns.plt.show()

 ![](https://s14-eu5.ixquick.com/cgi-bin/serveimage?url=http%3A%2F%2Ft0.gstatic.com%2Fimages%3Fq%3Dtbn%3AANd9GcQYogVQoXcb6iDm8h3OmyTga4BJT6ygVP2kTXq7_9ibJQJKuUba&sp=cbd598d3c54eb8a0b3a6fa73aac1aaf1&anticache=571068)
 
## DATA CLEANING

Once the dataset is filtered, and the listings are real and active, proper cleaning is the following stage:

1) Check duplicates, outliers.

2) Variable class conversion for following calculations.

3) Input of new variables

4) Creation of  a summary data frame broken down into districts for visualization purposes. 

In [20]:
# DUPLICATES
aib.duplicated().sum()

0

### Monetary variables

Conversion to EUR float in order to perform calculations
(All prices in dataset are in USD)

In [21]:
import re
aib['price']= aib['price'].str.replace('$','')
aib['price']= aib['price'].str.replace(',','')
aib['price'] = aib.price.astype(float)


In [22]:
aib['weekly_price']= aib['weekly_price'].str.replace('$','')
aib['weekly_price']= aib['weekly_price'].str.replace(',','')
aib['weekly_price'] = aib.weekly_price.astype(float)


In [23]:
aib['cleaning_fee']= aib['cleaning_fee'].str.replace('$','')
aib['cleaning_fee']= aib['cleaning_fee'].str.replace(',','')
aib['cleaning_fee'] = aib.cleaning_fee.astype(float)

In [24]:
aib['security_deposit']= aib['security_deposit'].str.replace('$','')
aib['security_deposit']= aib['security_deposit'].str.replace(',','')
aib['security_deposit'] = aib.security_deposit.astype(float)

In [25]:
# Extra people conversion to EUR
aib['extra_people'] = aib.extra_people.str.replace('$','')
aib['extra_people'] = aib.extra_people.astype(float)
aib['extra_people'] = aib.extra_people/ 1.0651

In [26]:
# Change prices to EUR.
# EURUSD exchange rate as of 4th april 2017: 1.0651
# https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/index.en.html
aib['price'] = aib.price * 1.0651
aib['weekly_price'] = aib.weekly_price / 1.0651
aib['cleaning_fee'] = aib.cleaning_fee / 1.0651
aib['security_deposit'] = aib.security_deposit/ 1.0651

In [ ]:
plt.plot(aib.price)
sns.plt.show()

### New Feature

In [27]:
# We add an extra column. Price per person to figure out average price per person in each neighbourhood
aib['Price_Person_Night'] = aib.price/aib.guests_included

### Outliers

Further investigation is required since maximum prices do not make sense in several neighbourhoods
(ie Argüelles).

In [28]:
# Search for outliers by neighbourhood and price
aibMed = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).price.agg(['max','mean','median','std'])

aibMed.sort_values(by='max',ascending=False)


max  \
neighbourhood_group_cleansed neighbourhood_cleansed      room_type                    
Moncloa - Aravaca            Argüelles                   Private room     8201.2700   
Salamanca                    Recoletos                   Entire home/apt  3396.6039   
Centro                       Universidad                 Entire home/apt  1065.1000   
                             Palacio                     Entire home/apt  1065.1000   
Chamberí                     Rios Rosas                  Private room     1065.1000   
Hortaleza                    Palomas                     Entire home/apt   905.3350   
Retiro                       Pacífico                    Entire home/apt   852.0800   
Chamartín                    Hispanoamérica              Entire home/apt   798.8250   
Moncloa - Aravaca            Ciudad Universitaria        Entire home/apt   798.8250   
Fuencarral - El Pardo        Peñagrande                  Private room      769.0022   
Villa de Vallecas            Casco Histórico de Vallecas Private room      769.0022   
Tetuán                       Bellas Vistas               Private room      762.6116   
Moncloa - Aravaca            Ciudad Universitaria        Private room      762.6116   
Tetuán                       Cuatro Caminos              Private room      761.5465   
Moncloa - Aravaca            Aravaca                     Private room      758.3512   
Fuencarral - El Pardo        Valverde                    Private room      753.0257   
                             El Goloso                   Private room      751.9606   
Chamberí                     Almagro                     Entire home/apt   745.5700   
Villaverde                   Los Angeles                 Private room      733.8539   
Chamartín                    Hispanoamérica              Private room      724.2680   
Moncloa - Aravaca            El Plantío                  Private room      692.3150   
Centro                       Cortes                      Private room      639.0600   
                             Justicia                    Entire home/apt   633.7345   
                             Cortes                      Entire home/apt   592.1956   
                             Sol                         Entire home/apt   585.8050   
Retiro                       Jerónimos                   Entire home/apt   570.8936   
Carabanchel                  San Isidro                  Entire home/apt   559.1775   
Centro                       Embajadores                 Entire home/apt   532.5500   
Chamberí                     Trafalgar                   Entire home/apt   532.5500   
Retiro                       Pacífico                    Private room      532.5500   
...                                                                             ...   
Usera                        Zofío                       Private room       26.6275   
Arganzuela                   Delicias                    Shared room        26.6275   
                             Chopera                     Shared room        24.4973   
Salamanca                    Guindalera                  Shared room        24.4973   
Villaverde                   San Cristobal               Private room       24.4973   
Hortaleza                    Valdefuentes                Shared room        23.4322   
Latina                       Cuatro Vientos              Private room       23.4322   
Villaverde                   San Cristobal               Shared room        21.3020   
Chamartín                    Nueva España                Shared room        21.3020   
Ciudad Lineal                Quintana                    Shared room        21.3020   
Tetuán                       Valdeacederas               Shared room        21.3020   
Fuencarral - El Pardo        Peñagrande                  Shared room        21.3020   
Puente de Vallecas           Entrevías                   Shared room        21.3020   
San Blas - Canillejas        Hellín                      Private room       21

 #### Manually Check id of suspect listings on Airbnb [website](https://www.airbnb.com/s/madrid/homes)
 

In [29]:
## ARGÛELLES
aib[(aib.neighbourhood_cleansed=='Argüelles')&(aib.room_type=='Private room')].sort_values(by='price',ascending=False)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
10828,1250842,https://www.airbnb.com/rooms/1250842,20170407214119,2017-04-08,Habitación Moncloa-AravacaArgüelles,NaN,Habitación amplia e iluminada. Luz natural 14 ...,Habitación amplia e iluminada. Luz natural 14 ...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,8201.27000
10869,18057261,https://www.airbnb.com/rooms/18057261,20170407214119,2017-04-08,European style apartment,"I say the cabin, convenient transportation, li...",NaN,"I say the cabin, convenient transportation, li...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,483.55540
10925,17961954,https://www.airbnb.com/rooms/17961954,20170407214119,2017-04-08,Intelligent bedrooms,Living room 65 inch LED TV can see the mainlan...,NaN,Living room 65 inch LED TV can see the mainlan...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,434.56080
10787,8828949,https://www.airbnb.com/rooms/8828949,20170407214119,2017-04-08,2Habitaciones privada- 2hab 2baños,Nuestra increíble casa en el centro de Madrid ...,"Diseño exquisito, casa preciosa y muy luminosa",Nuestra increíble casa en el centro de Madrid ...,none,Estamos en el centro de la ciudad a 5 minutos ...,...,t,NaN,NaN,f,moderate,f,f,2,0.17,127.81200
10752,16563606,https://www.airbnb.com/rooms/16563606,20170407214119,2017-04-08,Madrid.habitacion cuádruple centro,Habitación en él entró de Madrid . A 5 minutos...,NaN,Habitación en él entró de Madrid . A 5 minutos...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,7,NaN,91.59860
10764,8797816,https://www.airbnb.com/rooms/8797816,20170407214119,2017-04-08,Habitación privada y baño completo,"En el centro de Madrid, junto a la calle Princ...","Acogedor, silencioso y muy céntrico y bien com...","En el centro de Madrid, junto a la calle Princ...",none,"Barrio señorial, en el centro de madrid, con m...",...,t,NaN,NaN,t,moderate,f,f,2,0.06,85.20800
10746,5986943,https://www.airbnb.com/rooms/5986943,20170407214119,2017-04-08,Incredible double suite,Big double room (kingsize bed) with private ba...,"Its location, its high celings, peacefull place",Big double room (kingsize bed) with private ba...,none,"It is a very green neigborhood, with a big par...",...,t,NaN,NaN,t,flexible,f,f,3,1.98,79.88250
10914,14971934,https://www.airbnb.com/rooms/14971934,20170407214119,2017-04-08,NEW! Artistic double bedroom,"Our apartment is cozy, with a perfect location...","We want guests to feel at home and relax, so t...","Our apartment is cozy, with a perfect location...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,3,1.85,74.55700
10883,11103282,https://www.airbnb.com/rooms/11103282,20170407214119,2017-04-08,Alquilo apartamento!,Lindo espacioso,NaN,NaN,none,NaN,...,t,NaN,NaN,f,flexible,f,f,1,NaN,69.23150
10970,8299565,https://www.airbnb.com/rooms/8299565,20170407214119,2017-04-08,Great room! Great location!,"Hi, we are Nicolas and Diego, we are a gay cou...",NaN,"Hi, we are Nicolas and Diego, we are a gay cou...",none,NaN,...,t,NaN,NaN,f,flexible,f,f,1,1.02,53.25500


In [35]:
# The 8201 eur room is a  mistake since the listing is no longer on the website. Removed
aib = aib.drop(aib[aib.id==1250842].index)

In [36]:
# Recoletos
aib[(aib.neighbourhood_cleansed=='Recoletos')&(aib.room_type=='Entire home/apt')].sort_values(by='price',ascending=False)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
8027,2744574,https://www.airbnb.com/rooms/2744574,20170407214119,2017-04-08,Mansion with royal garden,This mansion has been a family home for genera...,NaN,This mansion has been a family home for genera...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,1,NaN,3396.603900
8154,16490785,https://www.airbnb.com/rooms/16490785,20170407214119,2017-04-08,Heima Home Serrano St. Penthouse - 5BR 4.5BT,This amazing 300m2 premium apartment at Calle ...,This spectacular penthouse apartment has 5 bed...,This amazing 300m2 premium apartment at Calle ...,none,"The position of the apartment is spectacular, ...",...,t,NaN,NaN,t,moderate,f,f,4,3.09,91.294286
8042,16379043,https://www.airbnb.com/rooms/16379043,20170407214119,2017-04-08,5bd Luxury Penthouse Prime location,5 Star Luxury Apartment 5 bedrooms. 3 ensuite...,This amazing penthouse which has been professi...,5 Star Luxury Apartment 5 bedrooms. 3 ensuite...,none,The flat is located in the prestigious aristoc...,...,t,NaN,NaN,t,moderate,f,f,1,1.00,633.734500
8021,13844318,https://www.airbnb.com/rooms/13844318,20170407214119,2017-04-08,Luxury Stay closest to Serrano,Apartment in the middle of the best neighbourh...,"200 m2 house, 4 bedrooms for 7 people","200 m2 house, 4 bedrooms for 7 people Best nei...",none,"Best neighborhood in madrid near best shops, r...",...,t,NaN,NaN,f,strict,f,f,1,0.48,616.692900
8057,18029659,https://www.airbnb.com/rooms/18029659,20170407214119,2017-04-08,Exclusive and comfortable - Marques del Duero II,Located in the exclusive neighborhood of Salam...,Marques del Duero II is on the third-floor ext...,Located in the exclusive neighborhood of Salam...,none,It is in one of the most distinguished areas o...,...,t,NaN,NaN,t,strict,f,f,77,NaN,614.562700
8117,16746286,https://www.airbnb.com/rooms/16746286,20170407214119,2017-04-08,Stunning 4 bedroom flat in Salamanca - Velazqu...,This wonderful four bedroom apartment is locat...,Located in the heart of one of the most exclus...,This wonderful four bedroom apartment is locat...,none,NaN,...,t,NaN,NaN,t,strict,f,f,77,NaN,592.195600
8074,9162754,https://www.airbnb.com/rooms/9162754,20170407214119,2017-04-08,Unique & exclusive flat for 6 -Marqués del Due...,Sat within the Spanish capital's most sought-a...,"For the rest, Marques del Duero I needs little...",Sat within the Spanish capital's most sought-a...,none,Surrounded by luxury in both its retail and re...,...,t,NaN,NaN,t,strict,f,f,77,0.74,570.893600
8056,16849985,https://www.airbnb.com/rooms/16849985,20170407214119,2017-04-08,Huge design flat in exclusive area - NuñezBalb...,Located in one of the most sought-after areas ...,Hallway: the apartment welcomes its tenants in...,Located in one of the most sought-after areas ...,none,Located in one of the most sought-after areas ...,...,t,NaN,NaN,t,strict,f,f,77,NaN,526.159400
8170,15142018,https://www.airbnb.com/rooms/15142018,20170407214119,2017-04-08,Super luxury 4 bedroom apartment in the best a...,"One of Europe’s most elegant city areas, just ...","Brand new, stylish and unbelievably spacious, ...","One of Europe’s most elegant city areas, just ...",none,This property is located in the heart of the B...,...,t,NaN,NaN,t,strict,f,f,11,0.36,67.456333
8029,9288847,https://www.airbnb.com/rooms/9288847,20170407214119,2017-04-08,Design in exclusive area - Núñez de Balboa VII,Superbly situated right in the heart of Madrid...,Located on the second floor of a traditional b...,Superbly situated right in the heart of Madrid...,none,"Just one block from Velazquez metro station, t...",...,t,NaN,NaN,t,strict,f,f,77,0.21,183.729750


In [37]:
# The listing with the highest price is an error. It is a house located in the outskirts of Madrid,
# not in  Recoletos. Removed
aib = aib.drop(aib[aib.id==2744574].index)

In [38]:
# Rios Rosas
aib[(aib.neighbourhood_cleansed=='Rios Rosas')&(aib.room_type=='Private room')].sort_values(by='price',ascending=False)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
10667,16199273,https://www.airbnb.com/rooms/16199273,20170407214119,2017-04-08,"Private Room in Rios Rosas, Madrid","My place is close to El Corte Ingles, Nuevos M...",NaN,"My place is close to El Corte Ingles, Nuevos M...",none,The neighborhood is really good. Quiet and cal...,...,t,NaN,NaN,f,moderate,f,f,1,NaN,1065.1000
10649,6474674,https://www.airbnb.com/rooms/6474674,20170407214119,2017-04-08,Habitación con Encanto,DISPONIBLE 1 DE JULIO!!!!!!Habitación matrimon...,La habitación es privada. Dispone de baño/coc...,DISPONIBLE 1 DE JULIO!!!!!!Habitación matrimon...,none,"Chamberí es un barrio muy bonito, de toda la v...",...,t,NaN,NaN,f,flexible,f,f,1,NaN,628.4090
10593,13755382,https://www.airbnb.com/rooms/13755382,20170407214119,2017-04-08,Habitación doble JULIO Y AGOSTO,Habitación completamente equipada con gran arm...,"Habitación fresquita, salón con aire acondicio...",Habitación completamente equipada con gran arm...,none,Barrio muy tranquilo pero al lado del mehoyo d...,...,t,NaN,NaN,f,flexible,f,f,1,NaN,420.7145
10664,18066296,https://www.airbnb.com/rooms/18066296,20170407214119,2017-04-08,Habitación doble en ático con terraza,Habitación doble en ático con terraza,NaN,Habitación doble en ático con terraza,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,95.8590
10605,4032408,https://www.airbnb.com/rooms/4032408,20170407214119,2017-04-08,"Bright, big & kosher-friendly",Spacious sunny room in traditional Chamberi. C...,The room is a true oasis of light and quietnes...,Spacious sunny room in traditional Chamberi. C...,none,The apartment is located in a safe area with a...,...,t,NaN,NaN,f,moderate,f,f,1,2.32,92.6637
10604,16943007,https://www.airbnb.com/rooms/16943007,20170407214119,2017-04-08,Excelente habitacion en Chamberì.,Ofrezco amplia habitacion con todas las comodi...,"Limpio, cómodo, económico.",Ofrezco amplia habitacion con todas las comodi...,none,Ubicado en la calle Ponzano q está muy de moda...,...,t,NaN,NaN,t,flexible,f,f,1,1.22,63.9060
10609,5112740,https://www.airbnb.com/rooms/5112740,20170407214119,2017-04-08,Room for rent - cool neighborhood!,"Apartamento de 2 habitaciones, muy luminosos! ...",Apartamento de 2 habitaciones (se alquila una ...,"Apartamento de 2 habitaciones, muy luminosos! ...",none,El barrio tiene todo lo que puedas necesitar: ...,...,t,NaN,NaN,f,moderate,f,f,1,NaN,63.9060
10591,1753278,https://www.airbnb.com/rooms/1753278,20170407214119,2017-04-08,Habitación doble en suite,Ofrecemos alojamiento para dos personas en nue...,NaN,Ofrecemos alojamiento para dos personas en nue...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,1,0.08,58.5805
10653,17548802,https://www.airbnb.com/rooms/17548802,20170407214119,2017-04-08,Well located in-suite bedroom,Super nice neighborhood. Walking distance to C...,"It's location, you will find everything you ne...",Super nice neighborhood. Walking distance to C...,none,NaN,...,t,NaN,NaN,t,strict,f,f,1,NaN,53.2550
10666,12201598,https://www.airbnb.com/rooms/12201598,20170407214119,2017-04-08,Habitación en piso inmejorable zona,1 habitaciones doble en Chamberí-Nuevos Minist...,NaN,1 habitaciones doble en Chamberí-Nuevos Minist...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,53.2550


In [39]:
# Again top 1 is a mistake. It is not available any more in the web. Removed
aib = aib.drop(aib[aib.id==16199273].index)

In [40]:
# Palacio
aib[(aib.neighbourhood_cleansed=='Palacio')&(aib.room_type=='Entire home/apt')].sort_values(by='price',ascending=False)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
957,10495669,https://www.airbnb.com/rooms/10495669,20170407214119,2017-04-08,Ático centrico y tranquilo / Atic,"Precioso ático totalmente reformado, en calle ...",Precioso atico justo en frente del Senado. Por...,"Precioso ático totalmente reformado, en calle ...",none,Es el madrid de los austrias La zona mas turís...,...,t,NaN,NaN,t,flexible,f,f,1,2.79,532.550000
455,17130010,https://www.airbnb.com/rooms/17130010,20170407214119,2017-04-08,LUJO ~La suite de Ópera~,SUITE AMPLIA Y LUJOSA EN LA ZONA MÁS EXCLUSIVA...,Se trata de un apartamento de gran calidad con...,SUITE AMPLIA Y LUJOSA EN LA ZONA MÁS EXCLUSIVA...,none,Ubicado en el corazón de Madrid el barrio de L...,...,t,NaN,NaN,t,flexible,f,f,1,0.67,479.295000
220,11343804,https://www.airbnb.com/rooms/11343804,20170407214119,2017-04-08,Heima Home Puerta del Sol - 6BR 4BT,El Heima Home Puerta del Sol es un espectacula...,El 5 Star Apartment Teatro Real / Sol es un lu...,El Heima Home Puerta del Sol es un espectacula...,none,El apartamento está situado en un precioso edi...,...,t,NaN,NaN,t,moderate,f,f,4,4.23,68.470714
31,13898710,https://www.airbnb.com/rooms/13898710,20170407214119,2017-04-08,Heima Home Cinema Puerta del Sol - 5BR 4.5BT,El Heima Home Cinema Puerta del Sol es un espe...,El Home Cinema Teatro Real / Sol es un lujoso ...,El Heima Home Cinema Puerta del Sol es un espe...,none,El apartamento está situado en un precioso edi...,...,t,NaN,NaN,t,moderate,f,f,4,2.09,60.862857
836,6119334,https://www.airbnb.com/rooms/6119334,20170407214119,2017-04-08,PlazaMayor/Sol-12-17Pax 6Bedr-+6Bathr + TV+A/C...,"Located 50m from the Royal Palace, the Cathedr...",Large FULLY AIR CONDITIONED apartment with 6 b...,"Located 50m from the Royal Palace, the Cathedr...",none,"Located in the heart of Madrid, 5 min. walking...",...,t,NaN,NaN,t,strict,f,f,1,2.50,426.040000
761,3984960,https://www.airbnb.com/rooms/3984960,20170407214119,2017-04-08,OPERA PENTHOUSE The Sibarist,This elegant and unique penthouse is located i...,"This penthouse of 140 square meters, is divide...",This elegant and unique penthouse is located i...,none,"The house is located in the heart of Madrid, i...",...,t,NaN,NaN,t,strict,f,f,9,0.65,426.040000
103,2116360,https://www.airbnb.com/rooms/2116360,20170407214119,2017-04-08,PALACIO DE REDONDILLA,"Espectacular apartamento de 3 dormitorios, con...","Distribuído en tres alturas, con amplios espac...","Distribuído en tres alturas, con amplios espac...",none,NaN,...,t,NaN,NaN,f,strict,t,t,6,0.72,399.412500
980,7740638,https://www.airbnb.com/rooms/7740638,20170407214119,2017-04-08,Unique views of the Royal Palace!,"Stately, sophisticated, stunning! Words cannot...","Some experiences are simply unique, and this i...","Stately, sophisticated, stunning! Words cannot...",none,The historic quarter of Habsburg Madrid is one...,...,t,VT-1957,NaN,t,strict,f,f,11,0.43,372.785000
596,2355655,https://www.airbnb.com/rooms/2355655,20170407214119,2017-04-08,Plaza Mayor 5 bedrooms 2 bathrooms,Beautiful apartment of 150 m2 with 5 bedrooms ...,It has 2 balconies overlooking the street to t...,Beautiful apartment of 150 m2 with 5 bedrooms ...,none,La Latina is one of the most traditional neigh...,...,t,NaN,NaN,t,strict,f,f,6,0.66,45.647143
442,2965016,https://www.airbnb.com/rooms/2965016,20170407214119,2017-04-08,Plaza Mayor 5 bedrooms 2 bathrooms,Beautiful apartment of 160 m2 with 5 bedrooms ...,It has 2 balconies overlooking the street to t...,Beautiful apartment of 160 m2 with 5 bedrooms ...,none,La Latina is one of the most traditional neigh...,...,t,NaN,NaN,t,strict,f,f,6,0.71,45.647143


In [41]:
# The listing with 1000 eur price has been delisted so we delete it.
aib = aib.drop(aib[aib.id==10495669].index)

In [42]:
# Ciudad Universitaria
aib[(aib.neighbourhood_cleansed=='Ciudad Universitaria')&(aib.room_type=='Private room')].sort_values(by='price',ascending=False)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
11221,17605002,https://www.airbnb.com/rooms/17605002,20170407214119,2017-04-08,Elegant Republic,Shanghai is located in the typical small alley...,Modification of the new neighborhood old villa...,Shanghai is located in the typical small alley...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,762.6116
11216,18105783,https://www.airbnb.com/rooms/18105783,20170407214119,2017-04-08,Comfortable two bedroom,"Housing for two rooms, refined decoration, the...",NaN,"Housing for two rooms, refined decoration, the...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,508.0527
11215,18059700,https://www.airbnb.com/rooms/18059700,20170407214119,2017-04-08,European style apartment,"I say the cabin, convenient transportation, li...",NaN,"I say the cabin, convenient transportation, li...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,491.0111
11264,18060882,https://www.airbnb.com/rooms/18060882,20170407214119,2017-04-08,European style apartment,"I say the cabin, convenient transportation, li...",NaN,"I say the cabin, convenient transportation, li...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,485.6856
11256,18104804,https://www.airbnb.com/rooms/18104804,20170407214119,2017-04-08,Cozy elevator room two,"The room is comfortable and warm, let you have...",NaN,"The room is comfortable and warm, let you have...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,473.9695
11220,17952035,https://www.airbnb.com/rooms/17952035,20170407214119,2017-04-08,Family Budget Apartments,Two bedrooms are equipped with air conditioner...,NaN,Two bedrooms are equipped with air conditioner...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,438.8212
11250,17953974,https://www.airbnb.com/rooms/17953974,20170407214119,2017-04-08,Hardcover Mediterranean,Two bedrooms are equipped with air conditioner...,NaN,Two bedrooms are equipped with air conditioner...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,437.7561
11239,17958269,https://www.airbnb.com/rooms/17958269,20170407214119,2017-04-08,Romantic garden,The room is equipped with an independent bathr...,NaN,The room is equipped with an independent bathr...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,434.5608
11218,17962836,https://www.airbnb.com/rooms/17962836,20170407214119,2017-04-08,Holiday Zone,"Room facilities, LCD TV, refrigerator, washing...",NaN,"Room facilities, LCD TV, refrigerator, washing...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,434.5608
11265,17959684,https://www.airbnb.com/rooms/17959684,20170407214119,2017-04-08,Free WIFI,"Autobudget Apartments, air conditioning, cable...",NaN,"Autobudget Apartments, air conditioning, cable...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,433.4957


In [43]:
# The top ten listings  are very weird offerings. Short Chinese description, no pictures,
# no reviews, no activity at all, and absurd high prices (375 -400 EUR per single room).
# A clear example of fake listings. We remove them
aib = aib.drop(aib[(aib.neighbourhood_cleansed=='Ciudad Universitaria')&(aib.room_type=='Private room')&(aib.price>150)&(aib.reviews_per_month.isnull())].index)

In [44]:
# Ciudad Universitaria
aib[(aib.neighbourhood_cleansed=='Ciudad Universitaria')&(aib.room_type=='Entire home/apt')].sort_values(by='price',ascending=False)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
11257,13647790,https://www.airbnb.com/rooms/13647790,20170407214119,2017-04-08,"Magnífica casa con jardín,piscina y pista de p...",Magnífica casa en una planta con una superfici...,NaN,Magnífica casa en una planta con una superfici...,none,NaN,...,t,NaN,NaN,f,strict,f,f,39,NaN,798.8250
11243,17711325,https://www.airbnb.com/rooms/17711325,20170407214119,2017-04-08,Victoria 4 bedrooms apartment,Spacious 4 bedroom house situated close to the...,Spacious 4 bedroom house situated close to the...,Spacious 4 bedroom house situated close to the...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,26.6275
11231,5563493,https://www.airbnb.com/rooms/5563493,20170407214119,2017-04-08,Maravillosa Casa en Barrio Moncloa,"Casa muy confortable, con todas las comodidade...",NaN,"Casa muy confortable, con todas las comodidade...",none,NaN,...,t,NaN,NaN,f,strict,f,f,1,0.21,138.4630
11246,17824570,https://www.airbnb.com/rooms/17824570,20170407214119,2017-04-08,Well fargo,The apartment in La Habana has 1 bedrooms and ...,The apartment in La Habana has 1 bedrooms and ...,The apartment in La Habana has 1 bedrooms and ...,none,NaN,...,f,NaN,NaN,t,strict,f,f,1,NaN,108.6402
11229,13203549,https://www.airbnb.com/rooms/13203549,20170407214119,2017-04-08,Spacious flat with views near the center,((SENSITIVE CONTENTS HIDDEN) translator) It is...,"It is comfortable, the views wide, it is near ...",((SENSITIVE CONTENTS HIDDEN) translator) It is...,none,"The building is located in a quiet area, not f...",...,t,NaN,NaN,f,flexible,f,f,3,0.77,106.5100
11255,10437387,https://www.airbnb.com/rooms/10437387,20170407214119,2017-04-08,Elegant and luminous flat in Madrid,Fully-equipped flat with large windows that pr...,The flat is fully-equipped with all the applia...,Fully-equipped flat with large windows that pr...,none,"Located in a quiet and safe area. However, you...",...,t,NaN,NaN,f,moderate,f,f,1,1.62,26.6275
11228,11748140,https://www.airbnb.com/rooms/11748140,20170407214119,2017-04-08,Fancy DUPLEX in jet set district,Enjoy our comfortable duplex in the jet-set di...,A duplex in Madrid´s most recognized residenti...,Enjoy our comfortable duplex in the jet-set di...,none,The neihborhood is considered one of the most ...,...,t,NaN,NaN,t,moderate,f,f,18,1.76,85.2080
11224,12808036,https://www.airbnb.com/rooms/12808036,20170407214119,2017-04-08,"MODERN,BRIGHT AND WELL LOCATED FLAT","Wonderful, comfortable, quiet and central apar...",This bright and quiet apartment has been just ...,"Wonderful, comfortable, quiet and central apar...",none,The apartment is located in the southern limit...,...,t,NaN,NaN,f,flexible,f,f,1,2.08,69.2315
11225,15786648,https://www.airbnb.com/rooms/15786648,20170407214119,2017-04-08,Apartamento Nuevo y luminoso,Te va a encantar mi lugar debido a el espacio ...,NaN,Te va a encantar mi lugar debido a el espacio ...,none,NaN,...,t,NaN,NaN,t,moderate,f,f,1,1.25,69.2315
11232,17568825,https://www.airbnb.com/rooms/17568825,20170407214119,2017-04-08,Cosy and sunny studio on top of Madrid,Cosy and sunny studio on top of Madrid....Acog...,Cosy and sunny studio on top of Madrid! 45 m2 ...,Cosy and sunny studio on top of Madrid....Acog...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,63.9060


In [45]:
# The second most expensive property is a mistake. It is located in Málaga, the fourth is not listed anymore. 
# Removed
aib = aib.drop(aib[(aib.id==17824570)&(aib.id==17711325)].index)


In [46]:
# HORTALEZA
aib[(aib.neighbourhood_cleansed=='Palomas')&(aib.room_type=='Entire home/apt')].sort_values(by='price',ascending=False)
# Prices are real listings.OK

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
12795,14495975,https://www.airbnb.com/rooms/14495975,20170407214119,2017-04-08,Escala Suites Chalet,Te va a encantar mi lugar debido a el espacio ...,NaN,Te va a encantar mi lugar debido a el espacio ...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,5,NaN,905.33500
12796,6433495,https://www.airbnb.com/rooms/6433495,20170407214119,2017-04-08,Luxury house with pool Close to airport & IFEMA,Luxury comfortable house with state of the art...,Unbelievable setting away from the speedy cent...,Luxury comfortable house with state of the art...,none,2 minutes away from Parque Juan Carlos I the b...,...,t,NaN,NaN,f,moderate,f,f,1,0.10,299.29310
12788,8517585,https://www.airbnb.com/rooms/8517585,20170407214119,2017-04-08,COSY & CHARMING flat with Wifi,Very close to the AIRPORT and right next to IF...,"The flat has a king bed in the bedroom, which ...",Very close to the AIRPORT and right next to IF...,none,The flat is located in a very quiet residentia...,...,t,NaN,NaN,f,moderate,f,f,1,1.74,36.74595
12797,10454527,https://www.airbnb.com/rooms/10454527,20170407214119,2017-04-08,Quiet private rooms close to Madrid Airport/ I...,Our shared house is simply minutes away from B...,NaN,Our shared house is simply minutes away from B...,none,The neighbourhood is extremely calm and peacef...,...,t,NaN,NaN,f,flexible,f,f,1,1.88,41.53890


In [47]:
# Fuencarral
aib[(aib.neighbourhood_group_cleansed=='Fuencarral - El Pardo')&(aib.room_type=='Private room')].sort_values(by='price',ascending=False)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
10732,17605703,https://www.airbnb.com/rooms/17605703,20170407214119,2017-04-08,The air garden room,Independent of the quality of all the houses a...,"The house has high power ion hair dryer, washi...",Independent of the quality of all the houses a...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,769.002200
11105,17839736,https://www.airbnb.com/rooms/17839736,20170407214119,2017-04-08,Can cook Online,"Hisense refrigerator, GREE air conditioning, M...",NaN,"Hisense refrigerator, GREE air conditioning, M...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,753.025700
11214,17841088,https://www.airbnb.com/rooms/17841088,20170407214119,2017-04-08,Can cook Online,"Hisense refrigerator, GREE air conditioning, M...",NaN,"Hisense refrigerator, GREE air conditioning, M...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,751.960600
11085,11607461,https://www.airbnb.com/rooms/11607461,20170407214119,2017-04-08,Casa bonita,"Lugares de interés: CASTILLO, actividades en f...",NaN,"Lugares de interés: CASTILLO, actividades en f...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,616.692900
11107,17966558,https://www.airbnb.com/rooms/17966558,20170407214119,2017-04-08,Comfortable spacious safe,Bedding to ensure a one for cleaning disinfect...,NaN,Bedding to ensure a one for cleaning disinfect...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,440.951400
11207,17965378,https://www.airbnb.com/rooms/17965378,20170407214119,2017-04-08,trouble taking static,"Two bedrooms a kitchen and a bathroom, all roo...",NaN,"Two bedrooms a kitchen and a bathroom, all roo...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,437.756100
10731,17952905,https://www.airbnb.com/rooms/17952905,20170407214119,2017-04-08,Hardcover family building,"24 hours hot water, a new shower, two new hang...",NaN,"24 hours hot water, a new shower, two new hang...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,436.691000
10704,17961687,https://www.airbnb.com/rooms/17961687,20170407214119,2017-04-08,Comfortable spacious safe,Bedding to ensure a one for cleaning disinfect...,NaN,Bedding to ensure a one for cleaning disinfect...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,434.560800
10688,17957775,https://www.airbnb.com/rooms/17957775,20170407214119,2017-04-08,Free WIFI,"Autobudget Apartments, air conditioning, cable...",NaN,"Autobudget Apartments, air conditioning, cable...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,433.495700
10690,17956357,https://www.airbnb.com/rooms/17956357,20170407214119,2017-04-08,Questa Bossa Mia,A total of 10 sets of units of the Mediterrane...,NaN,A total of 10 sets of units of the Mediterrane...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,433.495700


In [48]:
# Again expensive listings are fake. Same pattern than before, Chinese host ,no pictures,no reviews, no activity at all, 
# lack of proper description and absurd high prices (350 -400 EUR per single room).
# Most expensive listings delisted.
# We remove them
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Fuencarral - El Pardo')&(aib.room_type=='Private room')&(aib.price>350)&(aib.reviews_per_month.isnull())].index)


In [49]:
# Tetuan
aib[(aib.neighbourhood_group_cleansed=='Tetuán')&(aib.room_type=='Private room')].sort_values(by='price',ascending=False)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
9339,17836985,https://www.airbnb.com/rooms/17836985,20170407214119,2017-04-08,Can cook Online,"Hisense refrigerator, GREE air conditioning, M...",NaN,"Hisense refrigerator, GREE air conditioning, M...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,762.6116
9437,17613328,https://www.airbnb.com/rooms/17613328,20170407214119,2017-04-08,Fresh house rental,"Make a love yoga, love travel, love music, to ...","90, fine decoration houses, wooden floor board...","Make a love yoga, love travel, love music, to ...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,761.5465
9672,17961854,https://www.airbnb.com/rooms/17961854,20170407214119,2017-04-08,Family Budget Apartments,Two bedrooms are equipped with air conditioner...,NaN,Two bedrooms are equipped with air conditioner...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,434.5608
9375,17963109,https://www.airbnb.com/rooms/17963109,20170407214119,2017-04-08,Cozy warm room,"The kitchen is equipped with a gas cooker, fry...",NaN,"The kitchen is equipped with a gas cooker, fry...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,432.4306
9670,17954933,https://www.airbnb.com/rooms/17954933,20170407214119,2017-04-08,trouble taking static,"Two bedrooms a kitchen and a bathroom, all roo...",NaN,"Two bedrooms a kitchen and a bathroom, all roo...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,430.3004
9784,15763237,https://www.airbnb.com/rooms/15763237,20170407214119,2017-04-08,ALOJAMIENTO SANTA ROSA MADRID PRIMERO B,Mi alojamiento es bueno para viajeros de negoc...,NaN,Mi alojamiento es bueno para viajeros de negoc...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,4,NaN,191.7180
9738,15763726,https://www.airbnb.com/rooms/15763726,20170407214119,2017-04-08,ALOJAMIENTO SANTA ROSA MADRID SEGUNDO E,Lugares de interés: CERCA DEL CENTRO Y PARADA ...,NaN,Lugares de interés: CERCA DEL CENTRO Y PARADA ...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,4,0.36,191.7180
9785,17106378,https://www.airbnb.com/rooms/17106378,20170407214119,2017-04-08,ALOJAMIENTO SANTA ROSA MADRID PRIMERO A,Mi alojamiento es bueno para viajeros de negoc...,NaN,Mi alojamiento es bueno para viajeros de negoc...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,4,NaN,95.8590
9718,17207544,https://www.airbnb.com/rooms/17207544,20170407214119,2017-04-08,PRECIOSA HABITACIÓN EN LOFT ARTÍSTICO,Metro y Autobús.Aeropuerto a 10 min. en tren y...,Las mascotas cuentan con bol de agua y colchon...,Metro y Autobús.Aeropuerto a 10 min. en tren y...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,1,NaN,85.2080
9472,17032853,https://www.airbnb.com/rooms/17032853,20170407214119,2017-04-08,Room w/bath near Real Madrid Stadium,Room w/integrated bath in a Luxury Duplex of m...,Luxury duplex. Excellent location in the busin...,Room w/integrated bath in a Luxury Duplex of m...,none,The neighborhood is very well located in the b...,...,t,NaN,NaN,t,flexible,f,f,1,0.71,84.1429


In [50]:
# Again expensive listings are fake. Same pattern than before, Chinese host ,no pictures,no reviews, no activity at all, 
# lack of proper description and absurd high prices (350 -400 EUR per single room).
# Most expensive listings delisted.
# We remove them
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Tetuán')&(aib.room_type=='Private room')&(aib.price>350)&(aib.reviews_per_month.isnull())].index)


In [51]:
# Villa de Vallecas
aib[(aib.neighbourhood_group_cleansed=='Villa de Vallecas')&(aib.room_type=='Private room')].sort_values(by='price',ascending=False)
# Looks like a real listing.

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
13080,17612595,https://www.airbnb.com/rooms/17612595,20170407214119,2017-04-08,Supermarket loft apartment,The spatial pattern of loft simple all-match l...,My room surrounding and convenient transportat...,The spatial pattern of loft simple all-match l...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,769.0022
13057,126712,https://www.airbnb.com/rooms/126712,20170407214119,2017-04-08,hospedate,NaN,Interesados llamar mandar mensaje en la direcc...,Interesados llamar mandar mensaje en la direcc...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,1,NaN,85.2080
13084,1366943,https://www.airbnb.com/rooms/1366943,20170407214119,2017-04-08,2 rooms + bathroom with pool,NaN,Two rooms with souble beds and private bathroo...,Two rooms with souble beds and private bathroo...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,3,0.23,63.9060
13086,15876764,https://www.airbnb.com/rooms/15876764,20170407214119,2017-04-08,Habitación baño privado de apto nue,"Nuestro increíble, cómodo y nuevo apartamento ...","Todos están nuevos, No se huele nada de olor v...","Nuestro increíble, cómodo y nuevo apartamento ...",none,NaN,...,t,NaN,NaN,t,moderate,f,f,3,NaN,53.2550
13058,14313985,https://www.airbnb.com/rooms/14313985,20170407214119,2017-04-08,Habitacion Piscina Gym Sauna Padel,Se alquila Habitacion de matrimonio con Baño e...,NaN,Se alquila Habitacion de matrimonio con Baño e...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,1,NaN,53.2550
13061,549664,https://www.airbnb.com/rooms/549664,20170407214119,2017-04-08,SPACIOUS ROOM WITH BATHROOM,NaN,"SPACIOUS AND BRIGHT ROOM, WITH BUILT-IN WARDRO...","SPACIOUS AND BRIGHT ROOM, WITH BUILT-IN WARDRO...",none,NaN,...,t,NaN,NaN,t,moderate,f,f,1,0.04,53.2550
13078,15907899,https://www.airbnb.com/rooms/15907899,20170407214119,2017-04-08,Habitacion privada en apto nuevo,"Nuestro increíble, cómodo y nuevo apartamento ...",NaN,"Nuestro increíble, cómodo y nuevo apartamento ...",none,NaN,...,t,NaN,NaN,t,moderate,f,f,3,0.23,47.9295
13055,1529540,https://www.airbnb.com/rooms/1529540,20170407214119,2017-04-08,Habitación doble sin ruidos,NaN,H,H,none,NaN,...,t,NaN,NaN,f,flexible,f,f,1,0.42,37.2785
13096,4000092,https://www.airbnb.com/rooms/4000092,20170407214119,2017-04-08,PISO COMPARTIDO SOLO CHICAS,Piso compartido solo chicas estudiantes o jóv...,PISO COMPARTIDO SOLO CHICAS ESTUDIANTES O BEC...,PISO COMPARTIDO SOLO CHICAS ESTUDIANTES O BEC...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,1,0.03,31.9530
13056,17329862,https://www.airbnb.com/rooms/17329862,20170407214119,2017-04-08,Habitacion privada en apartamento con terraza,Es un piso con terraza en urbanización privada...,NaN,Es un piso con terraza en urbanización privada...,none,NaN,...,t,NaN,NaN,f,flexible,f,f,1,NaN,31.9530


In [52]:
# Chamartín
aib[(aib.neighbourhood_group_cleansed=='Chamartín')&(aib.room_type=='Private room')].sort_values(by='price',ascending=False)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Price_Person_Night
9114,17658219,https://www.airbnb.com/rooms/17658219,20170407214119,2017-04-08,Suitable for family friends,"Elevator apartment, indoor garden simple decor...","Public toilet, 24 hours uninterrupted constant...","Elevator apartment, indoor garden simple decor...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,724.26800
9231,17962303,https://www.airbnb.com/rooms/17962303,20170407214119,2017-04-08,Family Budget Apartments,Two bedrooms are equipped with air conditioner...,NaN,Two bedrooms are equipped with air conditioner...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,436.69100
8935,17962224,https://www.airbnb.com/rooms/17962224,20170407214119,2017-04-08,Free WIFI,"Autobudget Apartments, air conditioning, cable...",NaN,"Autobudget Apartments, air conditioning, cable...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,431.36550
8971,17962142,https://www.airbnb.com/rooms/17962142,20170407214119,2017-04-08,Questa Bossa Mia,A total of 10 sets of units of the Mediterrane...,NaN,A total of 10 sets of units of the Mediterrane...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,430.30040
8954,17625526,https://www.airbnb.com/rooms/17625526,20170407214119,2017-04-08,romantica,Mi alojamiento es bueno para parejas y familia...,NaN,Mi alojamiento es bueno para parejas y familia...,none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,266.27500
9059,9084826,https://www.airbnb.com/rooms/9084826,20170407214119,2017-04-08,Beautiful loft in Madrid,Great Location: 2 minutos to Metro Station (Pr...,NaN,Great Location: 2 minutos to Metro Station (Pr...,none,NaN,...,t,NaN,NaN,f,moderate,f,f,1,NaN,111.83550
8900,9517384,https://www.airbnb.com/rooms/9517384,20170407214119,2017-04-08,Piso lujo,Piso en una de las mejores zonas de Madrid. Ub...,Piso ubicado en una de las mejores zonas de Ma...,Piso en una de las mejores zonas de Madrid. Ub...,none,El piso esta ubicado en una de las mejores zon...,...,t,NaN,NaN,f,flexible,f,f,1,1.09,74.55700
9187,17210137,https://www.airbnb.com/rooms/17210137,20170407214119,2017-04-08,"Cerca del Bernabéu, piso soleado","Piso soleado muy cerca del estadio Bernabéu, g...",NaN,"Piso soleado muy cerca del estadio Bernabéu, g...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,74.55700
9176,15715749,https://www.airbnb.com/rooms/15715749,20170407214119,2017-04-08,habitacion cerca del bernabeu,"Muy buen anfitrion, Te puedo sugerir que hacer...",NaN,"Muy buen anfitrion, Te puedo sugerir que hacer...",none,NaN,...,t,NaN,NaN,t,flexible,f,f,1,NaN,74.55700
8983,6835710,https://www.airbnb.com/rooms/6835710,20170407214119,2017-04-08,Beautiful double bedroom,"This is my very own room, but since I will be ...",NaN,"This is my very own room, but since I will be ...",none,NaN,...,t,NaN,NaN,f,strict,f,f,1,0.14,74.55700


In [53]:
# Again expensive listings are fake. Same pattern than before, Chinese host ,no pictures,no reviews, no activity at all, 
# lack of proper description and absurd high prices (350 -400 EUR per single room).
# Most expensive listings delisted.
# We remove them
aib = aib.drop(aib[(aib.neighbourhood_group_cleansed=='Chamartín')&(aib.room_type=='Private room')&(aib.price>350)&(aib.reviews_per_month.isnull())].index)
aib.shape

(10077, 96)

In [ ]:
plt.plot(aib.price)
sns.plt.show()

In [ ]:
aib.to_csv('airbnb.csv')

### KEY FIGURES OF MADRID DISTRICTS

Summary of all important metrics of each neighbourhood in a single table. Long Format.

- Price.
- Max Price.
- Number of listings.
- Availability.
- Beds.
- Guests.
- Cleaning Fee.





In [54]:
# Prices
aib1 = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).price.mean()
aib1.rename(columns={'price':'Avg_Price'},inplace=True)

aib1a = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).price.max()
aib1a.rename(columns={'price':'Max_Price'},inplace=True)

aib1b = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).price.min()
aib1b.rename(columns={'price':'Min_Price'},inplace=True)

aib1c = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).price.median()
aib1c.rename(columns={'price':'Median_price'},inplace=True)
aib1a

,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,Max_Price
0,Arganzuela,Acacias,Entire home/apt,149.1140
1,Arganzuela,Acacias,Private room,53.2550
2,Arganzuela,Acacias,Shared room,106.5100
3,Arganzuela,Atocha,Entire home/apt,143.7885
4,Arganzuela,Atocha,Private room,74.5570
5,Arganzuela,Chopera,Entire home/apt,145.9187
6,Arganzuela,Chopera,Private room,53.2550
7,Arganzuela,Chopera,Shared room,24.4973
8,Arganzuela,Delicias,Entire home/apt,223.6710
9,Arganzuela,Delicias,Private room,85.2080


In [55]:
# Merge price metrics
# For accuracy reasons, the MEDIAN is used as measure of average price.
# Min price is ignored. 
dfs = [aib1a,aib1c]
pr = pd.merge(aib1a,aib1c, how='right',on=['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'])
pr

,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,Max_Price,Median_price
0,Arganzuela,Acacias,Entire home/apt,149.1140,69.23150
1,Arganzuela,Acacias,Private room,53.2550,26.62750
2,Arganzuela,Acacias,Shared room,106.5100,35.68085
3,Arganzuela,Atocha,Entire home/apt,143.7885,73.49190
4,Arganzuela,Atocha,Private room,74.5570,31.95300
5,Arganzuela,Chopera,Entire home/apt,145.9187,53.25500
6,Arganzuela,Chopera,Private room,53.2550,24.49730
7,Arganzuela,Chopera,Shared room,24.4973,24.49730
8,Arganzuela,Delicias,Entire home/apt,223.6710,63.90600
9,Arganzuela,Delicias,Private room,85.2080,31.95300


In [63]:
# Number of type of properties in each neighbourhood
aib2 = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).room_type.size().reset_index(name='Quantity')
aib2

,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,Quantity
0,Arganzuela,Acacias,Entire home/apt,45
1,Arganzuela,Acacias,Private room,53
2,Arganzuela,Acacias,Shared room,6
3,Arganzuela,Atocha,Entire home/apt,9
4,Arganzuela,Atocha,Private room,7
5,Arganzuela,Chopera,Entire home/apt,26
6,Arganzuela,Chopera,Private room,54
7,Arganzuela,Chopera,Shared room,1
8,Arganzuela,Delicias,Entire home/apt,41
9,Arganzuela,Delicias,Private room,51


In [56]:
# Average availability in the next 30 days in each area
aib3 = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).availability_30.mean()
aib3

,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,availability_30
0,Arganzuela,Acacias,Entire home/apt,8.622222
1,Arganzuela,Acacias,Private room,15.301887
2,Arganzuela,Acacias,Shared room,8.666667
3,Arganzuela,Atocha,Entire home/apt,5.222222
4,Arganzuela,Atocha,Private room,11.142857
5,Arganzuela,Chopera,Entire home/apt,8.884615
6,Arganzuela,Chopera,Private room,17.925926
7,Arganzuela,Chopera,Shared room,7.000000
8,Arganzuela,Delicias,Entire home/apt,8.707317
9,Arganzuela,Delicias,Private room,17.490196


In [57]:
# Average availability in the next 60 days in each area
aib4 = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).availability_60.mean()
aib4

,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,availability_60
0,Arganzuela,Acacias,Entire home/apt,21.977778
1,Arganzuela,Acacias,Private room,37.773585
2,Arganzuela,Acacias,Shared room,28.166667
3,Arganzuela,Atocha,Entire home/apt,18.777778
4,Arganzuela,Atocha,Private room,25.285714
5,Arganzuela,Chopera,Entire home/apt,25.807692
6,Arganzuela,Chopera,Private room,44.037037
7,Arganzuela,Chopera,Shared room,37.000000
8,Arganzuela,Delicias,Entire home/apt,20.878049
9,Arganzuela,Delicias,Private room,43.274510


In [58]:
# Many hosts do not charge cleaning fee. NaNs are replaced by 0 Eur
aib.cleaning_fee.isnull().sum()
aib['cleaning_fee']=aib['cleaning_fee'].replace(np.nan,0)
aib6 = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).cleaning_fee.mean()
aib6

,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,cleaning_fee
0,Arganzuela,Acacias,Entire home/apt,13.749361
1,Arganzuela,Acacias,Private room,5.526986
2,Arganzuela,Acacias,Shared room,0.000000
3,Arganzuela,Atocha,Entire home/apt,18.151660
4,Arganzuela,Atocha,Private room,2.950763
5,Arganzuela,Chopera,Entire home/apt,18.488694
6,Arganzuela,Chopera,Private room,5.546341
7,Arganzuela,Chopera,Shared room,0.000000
8,Arganzuela,Delicias,Entire home/apt,14.586973
9,Arganzuela,Delicias,Private room,3.608241


In [59]:
# Number of beds available in each neighbourhood
aib7 = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).beds.sum()
aib7

,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,beds
0,Arganzuela,Acacias,Entire home/apt,99.0
1,Arganzuela,Acacias,Private room,68.0
2,Arganzuela,Acacias,Shared room,11.0
3,Arganzuela,Atocha,Entire home/apt,26.0
4,Arganzuela,Atocha,Private room,8.0
5,Arganzuela,Chopera,Entire home/apt,57.0
6,Arganzuela,Chopera,Private room,90.0
7,Arganzuela,Chopera,Shared room,2.0
8,Arganzuela,Delicias,Entire home/apt,109.0
9,Arganzuela,Delicias,Private room,66.0


In [60]:
# Total NUmber of guests that may be hosted by area 
aib8 = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).accommodates.sum()
aib8

,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,accommodates
0,Arganzuela,Acacias,Entire home/apt,161
1,Arganzuela,Acacias,Private room,86
2,Arganzuela,Acacias,Shared room,12
3,Arganzuela,Atocha,Entire home/apt,39
4,Arganzuela,Atocha,Private room,12
5,Arganzuela,Chopera,Entire home/apt,102
6,Arganzuela,Chopera,Private room,109
7,Arganzuela,Chopera,Shared room,2
8,Arganzuela,Delicias,Entire home/apt,187
9,Arganzuela,Delicias,Private room,92


In [61]:
# Average price per person in each neighbourhood
aib9 = aib.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],as_index=False).Price_Person_Night.mean()
aib9

,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,Price_Person_Night
0,Arganzuela,Acacias,Entire home/apt,52.352764
1,Arganzuela,Acacias,Private room,28.446208
2,Arganzuela,Acacias,Shared room,47.396950
3,Arganzuela,Atocha,Entire home/apt,37.870222
4,Arganzuela,Atocha,Private room,35.680850
5,Arganzuela,Chopera,Entire home/apt,41.801761
6,Arganzuela,Chopera,Private room,24.408542
7,Arganzuela,Chopera,Shared room,24.497300
8,Arganzuela,Delicias,Entire home/apt,49.763117
9,Arganzuela,Delicias,Private room,31.201165


### VISUALIZATION DATASET (R)

In [64]:
# Merge previuos dfs to have the expected summary data frame
dfs = [pr,aib2,aib3,aib4,aib7,aib8,aib9]
aibsum = reduce(lambda left,right: pd.merge(left,right, on =['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type']),dfs)
aibsum.head()

,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,Max_Price,Median_price,Quantity,availability_30,availability_60,beds,accommodates,Price_Person_Night
0,Arganzuela,Acacias,Entire home/apt,149.1140,69.23150,45,8.622222,21.977778,99.0,161,52.352764
1,Arganzuela,Acacias,Private room,53.2550,26.62750,53,15.301887,37.773585,68.0,86,28.446208
2,Arganzuela,Acacias,Shared room,106.5100,35.68085,6,8.666667,28.166667,11.0,12,47.396950
3,Arganzuela,Atocha,Entire home/apt,143.7885,73.49190,9,5.222222,18.777778,26.0,39,37.870222
4,Arganzuela,Atocha,Private room,74.5570,31.95300,7,11.142857,25.285714,8.0,12,35.680850


In [ ]:
# Aibsum saved as csv. We will use this data frame in R to visualize data in a Madrid map
aibsum.to_csv('aibsuma.csv')

In [65]:
# Transform dataframe to wide format in order to have a breakdown of figures in each area.
h2 = pd.pivot_table(aibsum,index = ['neighbourhood_group_cleansed','neighbourhood_cleansed','room_type'],values = ['Max_Price','Median_price','Quantity','availability_30',
                                'availability_60','beds','accommodates','Price_Person_Night'])
h2

Max_Price  \
neighbourhood_group_cleansed neighbourhood_cleansed       room_type                    
Arganzuela                   Acacias                      Entire home/apt   149.1140   
                                                          Private room       53.2550   
                                                          Shared room       106.5100   
                             Atocha                       Entire home/apt   143.7885   
                                                          Private room       74.5570   
                             Chopera                      Entire home/apt   145.9187   
                                                          Private room       53.2550   
                                                          Shared room        24.4973   
                             Delicias                     Entire home/apt   223.6710   
                                                          Private room       85.2080   
                                                          Shared room        26.6275   
                             Imperial                     Entire home/apt   117.1610   
                                                          Private room       85.2080   
                             Legazpi                      Entire home/apt   213.0200   
                                                          Private room       52.1899   
                                                          Shared room        20.2369   
                             Palos de Moguer              Entire home/apt   266.2750   
                                                          Private room      158.6999   
                                                          Shared room        12.7812   
Barajas                      Aeropuerto                   Entire home/apt    42.6040   
                                                          Private room       68.1664   
                             Alameda de Osuna             Entire home/apt   101.1845   
                                                          Private room      106.5100   
                             Casco Histórico de Barajas   Entire home/apt   319.5300   
                                                          Private room       47.9295   
                             Corralejos                   Private room       55.3852   
                             Timón                        Entire home/apt   105.4449   
                                                          Private room      436.6910   
Carabanchel                  Abrantes                     Entire home/apt    56.4503   
                                                          Private room       74.5570   
...                                                                              ...   
Usera                        Moscardó                     Private room       34.0832   
                                                          Shared room        17.0416   
                             Orcasitas                    Private room       21.3020   
                             Orcasur                      Private room       85.2080   
                             Pradolongo                   Entire home/apt    90.5335   
                                                          Private room       56.4503   
                             San Fermín                   Entire home/apt    85.2080   
                                                          Private room       31.9530   
                             Zofío                        Entire home/apt    77.7523   
                                                          Private room       26.6275   
Vicálvaro                    Ambroz                       Private room       19.1718   
                             Casco Histórico de Vicálvaro Entire home/apt   200.2388   
                                                          Private room       35.1483   
Villa de Vallecas          